<a href="https://colab.research.google.com/github/isabelladegen/ComputationalLogic/blob/prolexa-plus/Demo_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Prolexa Coursework Demo Notebook - Isabella

This notebook is used to:

1. Demonstrate the new functionality added to Prolexa
2. Test the existing functionalty to ensure I didn't break it

The setup of Prolexa is copied from the Prolexa_Plus_Demo_Notebook.ipynb. However this notebook does not use Prolexa Plus and instead direclty calls Prolexa. Due to not using Prolexa Plus, **proper nouns** cannot be added to prolexa.  New rules about existing proper nouns can still be added.

### Notebook Structure:

- Installation & Setup: 
  - installs prolog, prolexa and define query functions
- Testing Existing and New Functionality: 
  - defines test helper functions ```test()``` & ```cleanup()```
  - collection of test queries for the existing and new functionality
- Tru yourself! 
  - widget to interact with Prolexa and try out the changes





### Prolexa's starting knowledge:
**Proper Nouns:** see [prolexa_grammar.pl](https://github.com/isabelladegen/ComputationalLogic/blob/prolexa-plus/prolexa/prolog/prolexa_grammar.pl)
>```
proper_noun(s,tweety) --> [tweety].
proper_noun(s,peter) --> [peter].
proper_noun(s,opus) --> [opus].
proper_noun(s,peep) --> [peep].
```

**Rules:** see [prolexa.pl](https://github.com/isabelladegen/ComputationalLogic/blob/prolexa-plus/prolexa/prolog/prolexa.pl)

> ```
  %some intial stored rules
  stored_rule(1,[(mortal(X):-human(X))]).
  stored_rule(1,[(human(peter):-true)]).
  %additional rules for default reasoning
  stored_rule(1,[(default(fly(X):-bird(X)))]).
  stored_rule(1,[(not fly(X):-penguin(X))]).
  stored_rule(1,[(bird(X):-penguin(X))]).
  stored_rule(1,[(penguin(opus):-true)]).
  stored_rule(1,[(bird(peep):-true)]).
  ```

**Grammar:** see [prolexa_grammar.pl](https://github.com/isabelladegen/ComputationalLogic/blob/prolexa-plus/prolexa/prolog/prolexa_grammar.pl)

>  ```
  pred(human, 1,[a/human,n/human]).
  pred(mortal, 1,[a/mortal,n/mortal]).
  pred(bird, 1,[n/bird]).
  pred(penguin, 1,[n/penguin]).
  pred(swan, 1,[n/swan]).
  pred(fly, 1,[v/fly]).
  pred(sing, 1,[v/sing]).
  ```






# Installation & Setup

### Install SWI-Prolog

*Copied from Prolexa_Plus_Demo_Notebook.ipynb*

In [1]:
!apt-get install swi-prolog -qqq > /dev/null

Extracting templates from packages: 100%


### Installing Prolexa

Copied from Prolexa_Plus_Demo_Notebook.ipynb added **--upgrade** to always pull changes from the git repostiory
*Same error as forked branch https://github.com/simply-logical/ComputationalLogic/*

In [2]:
!yes | pip install --upgrade git+https://github.com/isabelladegen/ComputationalLogic/ -qqq > /dev/null

  ERROR: Failed building wheel for prolexa
  DEPRECATION: prolexa was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. You can find discussion regarding this at https://github.com/pypa/pip/issues/8368.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
markdown 3.3.6 requires importlib-metadata>=4.4; python_version < "3.10", but you have importlib-metadata 3.10.1 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


### Helper Functions

- Functions to call Prolexa instead of Prolexa Plus
- Query syntax:  ```query('string')```


In [3]:
import os
from pyswip import Prolog
import site
 
def find_prolog_path():
  packages_paths = site.getsitepackages()
  for ppath in packages_paths:
    if 'prolexa' in os.listdir(ppath):
      return os.path.join(ppath, 'prolexa/prolog')
    else:
      raise Exception(f'\'prolexa\' package was not found in {packages_path}. Did you install it?')
                
pl = Prolog()

def initialise_prolexa():
  pl.consult(os.path.join(find_prolog_path(), 'prolexa.pl'))

def query(input): #also inspired from prolexa plus but adopted
  initialise_prolexa()
  libPrefix = 'prolexa:'
  generator = pl.query(libPrefix + handle_utterance_str(input))
  answer = list(generator)[0]['Output']
  if isinstance(answer, str):
    return answer
  return str(answer, 'utf-8')

def handle_utterance_str(text): #this is from the meta_grammar from prolexa plus!
    if text[0] != "'" and text[0] != '"' :
        text = f'"{text}"'

    text = text.replace('"', '\"')
    text = text.replace("'", '\"')

    return 'handle_utterance(1,{},Output)'.format(text)

In [4]:
query('Tell me all')

'every human is mortal. peter is human. some birds fly. penguins dont fly. every penguin is a bird. opus is a penguin. peep is a bird'

# Testing Existing and New Functionality

You can write a new test like this: ```test('query', 'expected response')```. The test passes if the response of the query matches the expected response and fails if not.

You can call ```cleanup()``` to get back to the initial state of Prolexa, e.g after adding knowledge in a test query or via a normal query.

In [5]:
def test(input, expected_answer):
  # print(f'Query: {input}')
  actual_answer = query(input)
  # print(f'Answer: {actual_answer}\n')
  assert actual_answer == expected_answer, 'Expected Answer:  ' + expected_answer + '\n Actual answer: ' + actual_answer

def cleanup():
  query('forget everything')
  initialise_prolexa()


## Test Existing Functionality


In [6]:
cleanup() #start clean, in case other tests added knowledge
#Check whats known in the stored rules
test('Spill the beans', 'every human is mortal. peter is human. some birds fly. penguins dont fly. every penguin is a bird. opus is a penguin. peep is a bird')

#Tell me about... queries
test('Tell me all', 'every human is mortal. peter is human. some birds fly. penguins dont fly. every penguin is a bird. opus is a penguin. peep is a bird') #extened with new rules added to prolexa
test('tell me about unknownnoun', 'I heard you say,  tell me about unknownnoun , could you rephrase that please?')
test('tell me about tweety', 'I know nothing about tweety')
test('tell me about peter', 'peter is human. peter is mortal')
test('Spill the beans', 'every human is mortal. peter is human. some birds fly. penguins dont fly. every penguin is a bird. opus is a penguin. peep is a bird')

#Is... queries
test('Is unkownnoun human', 'I heard you say,  Is unkownnoun human , could you rephrase that please?')
test('Is tweety a bird', 'Sorry, I don\'t think this is the case')
test('Is peter mortal', 'peter is mortal')
test('Is peter human', 'peter is human')
test('Who is human', 'peter is human')
test('Who is a bird', 'opus is a bird')
test('Does tweety fly', 'Sorry, I don\'t think this is the case')

#Explain... queries
test('Explain why unknownnoun is mortal', 'I heard you say,  Explain why unknownnoun is mortal , could you rephrase that please?') #unkown words don't work
test('Explain why tweety is a bird','Sorry, I don\'t think this is the case')
test('Explain why peter is mortal', 'peter is human; every human is mortal; therefore peter is mortal')

#Adding new rules and forgeting rules
test('Peter is mortal', 'I already knew that Peter is mortal')
test('Tweety is a bird', 'I will remember that Tweety is a bird')
test('Is tweety a bird', 'tweety is a bird')
test('Tell me all about tweety', 'tweety is a bird. tweety flies') 
test('Forget that tweety is a bird', 'I erased it from my memory')
test('tell me all about tweety', 'I know nothing about tweety')
test('Is tweety a bird', 'Sorry, I don\'t think this is the case')
test('Does tweety fly', 'Sorry, I don\'t think this is the case')
cleanup()


## Test New Functionality


### Default Reasoning Questions & Explanation

- Introduced ability to define rules with exceptions as **default** rules:
```stored_rule(1,[(default(fly(X):-bird(X)))]).``` These will be found last for a proof and will be explained as: *'some ns do y'* instead of *'every n is y'* or *'n do y"*
- Introduced negation **not** in rules to define exceptions: ```stored_rule(1,[(not fly(X):-penguin(X))]).``` which translates to *'penguins dont fly'* Question about a proper noun also include negated rules. *Note that don't is written as dont as I've not added and escaped the apostrophe. To query 'dont' and 'doesnt' has to be used.*
- Extended explain and proof logic to explain default rules as 'some ns...' and to explain negated rules as *'ns dont do y'*
- Extended explain logic to be able to explain negative statments: 'Explain why opus doesnt fly' &rarr; 'opus is a penguin; penguins dont fly; therefore opus doesnt fly'

The tests below demonstrate these features:

In [15]:
cleanup()
#Check what's already known about tweety, peep and opus in the rulebase before we start adding
test('Spill the beans', 'every human is mortal. peter is human. some birds fly. penguins dont fly. every penguin is a bird. opus is a penguin. peep is a bird')

#Queries
test('Tell me about opus', 'opus is a bird. opus is a penguin. opus doesnt fly') #reports on negated rules from other truths: not(fly(X):-penguin(X)) & penguin(opus):-true
test('Tell me about peep', 'peep is a bird. peep flies')
test('Is opus a bird', 'opus is a bird')
test('Is peep a bird', 'peep is a bird')
test('Is opus a penguin', 'opus is a penguin')
test('Is peep a penguin', 'Sorry, I don\'t think this is the case')
test('Does opus fly', 'Sorry, I don\'t think this is the case')
test('Does peep fly','peep flies')

#Explanations
test('Explain why peep flies', 'peep is a bird; some birds fly; therefore peep flies') #explaining using default rules for flying
test('Explain why opus flies', 'Sorry, I don\'t think this is the case') #cannot explain that opus flies (cause opus doesn't fly)
test('Explain why opus doesnt fly', 'opus is a penguin; penguins dont fly; therefore opus doesnt fly') #can explain that opus doesn't fly

#Check what's known now
test('Spill the beans', 'every human is mortal. peter is human. some birds fly. penguins dont fly. every penguin is a bird. opus is a penguin. peep is a bird')
cleanup()


### Creating new rules including **Default Rules**
- You can add **default rules** about nouns like this: 'Some n do y' adds the default rule ```stored_rule(1,[(default(y(X):n(X))]).```
- You can add **negated rules**: 'Ns dont do y' adds ```stored_rule(1,[(not y(X):-n(X))])```
- You can continue to add normal rules about nouns: 'All ns do y' adds ```stored_rule(1,[y(X):-n(X)])```
- For proper nouns you can continue to add rules via 'X does y'
- If you add the same default rule multiple times it will not add it multiple times but instead also results in *'I already knew that'*

**Note you cannot extend the vocabulary. The vobabulary has the verbs to sing and to fly and the proper noun Swan added for testing. There are no predefined rules about Swans and singing. There's a rule defined that Penguins dont fly.*

***Keep  in mind that the rules get added to the bottom some care about ordering has to be taken.* 


In [8]:
cleanup()
#Check what's already known about tweety, peep and opus in the rulebase before we start adding
test('Spill the beans', 'every human is mortal. peter is human. some birds fly. penguins dont fly. every penguin is a bird. opus is a penguin. peep is a bird')
test('Tell me about tweety', 'I know nothing about tweety')
test('Tell me about peep', 'peep is a bird. peep flies')
test('Tell me about opus', 'opus is a bird. opus is a penguin. opus doesnt fly')

#Create new Default and normal rules
test('Some birds sing', 'I will remember that Some birds sing') #adds a new default: stored_rule(1,[(default(sing(X):-bird(X)))])
test('Swans dont sing', 'I will remember that Swans dont sing') #adds a new rule: stored_rule(1,[(not sing(X):-swan(X))])
test('All swans are birds', 'I will remember that All swans are birds') #adds a new rule: stored_rule(1,[(bird(X):-swan(X))])
test('Tweety is a swan', 'I will remember that Tweety is a swan') #adds a new rule: stored_rule(1,[(swan(tweety):-true)])
#all that's known now
test('Spill the beans', 'every human is mortal. peter is human. some birds fly. penguins dont fly. every penguin is a bird. opus is a penguin. peep is a bird. some birds sing. swans dont sing. every swan is a bird. tweety is a swan')

#Ensure that adding the rules twice does not add the rules twice - (Idempotence)
test('Some birds sing', 'I already knew that Some birds sing') #also works for default rules now
test('Swans dont sing', 'I already knew that Swans dont sing') 
test('All swans are birds', 'I already knew that All swans are birds')
test('Tweety is a swan', 'I already knew that Tweety is a swan')
#all that's known now
test('Spill the beans', 'every human is mortal. peter is human. some birds fly. penguins dont fly. every penguin is a bird. opus is a penguin. peep is a bird. some birds sing. swans dont sing. every swan is a bird. tweety is a swan')

#Show what is known about all the different birds with the new rules
test('Tell me about tweety', 'tweety is a bird. tweety is a swan. tweety flies. tweety doesnt sing')
test('Tell me about peep', 'peep is a bird. peep flies. peep sings')
test('Tell me about opus', 'opus is a bird. opus is a penguin. opus doesnt fly. opus sings')

#Demonstrate explanations
test('Explain why peep sings', 'peep is a bird; some birds sing; therefore peep sings')
test('Explain why tweety doesnt sing', 'tweety is a swan; swans dont sing; therefore tweety doesnt sing') 
test('Explain why tweety flies', 'tweety is a swan; every swan is a bird; some birds fly; therefore tweety flies') 
test('Explain why opus sings', 'opus is a penguin; every penguin is a bird; some birds sing; therefore opus sings')

#Demonstrate questions
test('Does tweety sing', 'Sorry, I don\'t think this is the case')
test('Does opus sing', 'opus sings')
test('Does peep sing', 'peep sings')

test('Does tweety fly', 'tweety flies')
test('Does opus fly', 'Sorry, I don\'t think this is the case')
test('Does peep fly', 'peep flies')

test('Is tweety a bird', 'tweety is a bird')
test('Is opus a bird', 'opus is a bird')
test('Is peep a bird', 'peep is a bird')

### Negation
Negation has been implemented as part of default reasoning as I needed the ability to define exceptions:
- You can add negations about a proper noun: 'Peter doesn't fly' adds a new rule ```stored_rule(1,[(not fly(peter):-true)])```
- If a negated rule has been defined, then that rule will be included in explanations *'Explain why...'* as well as when listing all that's known *'Tell me about...'*

In [18]:
cleanup()
#check what's already in the rulebase
test('Tell me about Peter', 'peter is human. peter is mortal')

#Create a negated fact about Peter
test('Peter doesnt fly', 'I will remember that Peter doesnt fly') #adds a new rule: stored_rule(1,[(not fly(peter):-true)])
test('Peter doesnt fly', 'I already knew that Peter doesnt fly') #ensure calling again doesn't add another rule

#Questions
test('Does Peter fly', 'Sorry, I don\'t think this is the case')

#check what's now known about Peter
test('Tell me about Peter', 'peter is human. peter is mortal. peter doesnt fly')
test('Explain why Peter doesnt fly', 'peter doesnt fly; therefore peter doesnt fly') 

# Try yourself!

This allows you to interact with Prolexa. The "Reset Rules" button resets back to the initial state. This has been adopted from Prolexa Plus Demo Notebook.ipynb to interact directly with Prolexa.

Remember to type **dont** and **doesnt** instead of don't and doesn't.

In [55]:
cleanup()
import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import display, HTML

w_textbox = widgets.Text(
    value='Tell me all',
    placeholder='Enter a query',
    description='Question:',
    disabled=False,
    layout=Layout(width='900px', height='50px')
)
ask_button = widgets.Button(
    description='Ask',
    button_style='info',
    layout=Layout(margin='4px 0px 0px 90px')
)
cleanup_button = widgets.Button(
    description='Reset Rules',
    layout=Layout(margin='4px 0px 0px 5px')
)
w_out = widgets.Output(layout={'border': '1px solid black', 'margin': '10px', 'width':'1000px', 'word-break': 'break-all'})

def clicked_cleanup(obj):
    cleanup()

def main(obj):
    question = w_textbox.value
    answer = query(question)
    w_textbox.value = ''
    with w_out:
        print('?', question)
        print(answer)

# bind event handler to UI control
ask_button.on_click(main)
cleanup_button.on_click(clicked_cleanup)
# render UI controls in a vertical box
widgets.VBox([w_textbox, widgets.HBox([ask_button, cleanup_button]), w_out])